<a href="https://colab.research.google.com/github/RedaElmar/DWBI_ETL/blob/master/Scrapping%20and%20cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# County Level Unemployement Data Scraper 2010-2019

In [67]:
#--------------------------------------
#   Area Unemployment statistics: https://download.bls.gov/pub/time.series/la/
#--------------------------------------

import pandas as pd
import requests

#------------------------------------------------------
# Download and save .TXT files from BLS website

BLS_url = 'https://download.bls.gov/pub/time.series/la/'

filenames = ['la.area','la.data.0.CurrentU10-14','la.data.0.CurrentU15-19']

for xx in filenames:
    dls = BLS_url+xx
    resp = requests.get(dls)

    output = open(xx+'.txt', 'wb')
    output.write(resp.content)
    output.close()
#MApping Area Code with County Name 
df_areas = pd.read_table('la.area.txt')
df_areas = df_areas[['area_code', 'area_text']]

df_areas = df_areas.loc[df_areas['area_code'].str.contains('CN')]
df_areas.reset_index(drop=True, inplace=True)

df_areas.columns = ['area_code', 'countyname']

tmp = df_areas['countyname'].str.split(', ', expand=True)
df_areas['countyname'] = tmp[0]
df_areas['state'] = tmp[1]

df_areas['area_code'] = df_areas['area_code'].map(lambda x: x.strip())
df_areas['countyname'] = df_areas['countyname'].map(lambda x: x.strip())

#------------------------------------------------------

In [68]:

def get_BLS_county_data(BLS_data_path, df_areas):

    # Import area information
    col_types = {'series_id': str, 'year': int, 'period': str, 'value': str, 'footnote_codes': str}
    df_bls_county = pd.read_table(BLS_data_path, dtype=col_types)
    
    # Remove white space from code..
    df_bls_county['series_id'] = df_bls_county['series_id                     '].map(lambda x: x.strip())
    df_bls_county['value'] = df_bls_county['       value'].apply(pd.to_numeric, errors='coerce')

    # Get variable code
    df_bls_county['var_code'] = df_bls_county['series_id'].str[-2:]
    # Get area code
    df_bls_county['series_id'] = df_bls_county['series_id'].astype(str).str[3:].str[:-2]
    # Get FIPS code (as string to preserve initial zeros)
    df_bls_county['FIPS'] = df_bls_county['series_id'].str[2:7]
    # Only keep rows corresponding to counties
    df_bls_county = df_bls_county.loc[df_bls_county['series_id'].str.contains('CN')]

    # Drop columns, reset index
    df_bls_county = df_bls_county[['series_id','year','period','value','var_code','FIPS']]
    df_bls_county.reset_index(drop=True, inplace=True)

    # Rename codes with variable names, rename columns
    df_bls_county['var_code'] = df_bls_county['var_code'].map({'03': 'Unemployment_Rate', '04': 'Unemployment',
                                                                 '05': 'Employment', '06': 'Labor_Force'})
    df_bls_county.columns = ['area_code', 'year', 'month', 'value','variable_name', 'FIPS']

    # Drop month 13 (year average)
    df_bls_county = df_bls_county.loc[df_bls_county['month']!='M13']
    df_bls_county['month'] = pd.to_numeric(df_bls_county['month'].str[1:])

    df_bls_county = pd.merge(df_bls_county, df_areas, how='inner', on='area_code')

    df_bls_county = df_bls_county.pivot_table(values='value', index=['area_code', 'FIPS', 'state', 'countyname','year', 'month'], columns='variable_name')
    df_bls_county.reset_index(inplace=True)
    df_bls_county.columns.name = None
    print('Done!')

    return df_bls_county


In [69]:
df_unemp_10_14 = get_BLS_county_data('la.data.0.CurrentU10-14.txt', df_areas)

Done!


In [70]:
df_unemp_10_14.tail(30)

,area_code,FIPS,state,countyname,year,month,Employment,Labor_Force,Unemployment,Unemployment_Rate
193050,CN7215300000000,72153,PR,Yauco Municipio,2012,7,9366.0,11863.0,2497.0,21.0
193051,CN7215300000000,72153,PR,Yauco Municipio,2012,8,9067.0,11409.0,2342.0,20.5
193052,CN7215300000000,72153,PR,Yauco Municipio,2012,9,9225.0,11532.0,2307.0,20.0
193053,CN7215300000000,72153,PR,Yauco Municipio,2012,10,9133.0,11482.0,2349.0,20.5
193054,CN7215300000000,72153,PR,Yauco Municipio,2012,11,9057.0,11299.0,2242.0,19.8
193055,CN7215300000000,72153,PR,Yauco Municipio,2012,12,9109.0,11423.0,2314.0,20.3
193056,CN7215300000000,72153,PR,Yauco Municipio,2013,1,9206.0,11910.0,2704.0,22.7
193057,CN7215300000000,72153,PR,Yauco Municipio,2013,2,8940.0,11339.0,2399.0,21.2
193058,CN7215300000000,72153,PR,Yauco Municipio,2013,3,8935.0,11220.0,2285.0,20.4
193059,CN7215300000000,72153,PR,Yauco Municipio,2013,4,8855.0,10997.0,2142.0,19.5


In [71]:
df_unemp_10_14.columns

Index(['area_code', 'FIPS', 'state', 'countyname', 'year', 'month',
       'Employment', 'Labor_Force', 'Unemployment', 'Unemployment_Rate'],
      dtype='object')

Save Scrapped Data To Csv file

In [72]:
df_unemp_10_14 = get_BLS_county_data('la.data.0.CurrentU10-14.txt', df_areas)
df_unemp_15_19 = get_BLS_county_data('la.data.0.CurrentU15-19.txt', df_areas)

df_unemp_county = df_unemp_10_14
df_unemp_county = df_unemp_county.append(df_unemp_15_19)

df_unemp_county = df_unemp_county.sort_values(by=['area_code', 'year', 'month'], axis=0)
df_unemp_county[['FIPS', 'state', 'countyname', 'year', 'month','Employment', 'Labor_Force', 'Unemployment', 'Unemployment_Rate']].to_csv('BLS_county_employment.csv', index=False)

Done!
Done!


In [73]:
!ls -l -sh

total 273M
 22M -rw-r--r-- 1 root root  22M Jan  3 23:01 BLS_county_employment.csv
252K -rw-r--r-- 1 root root 250K Jan  3 20:19 div.csv
272K -rw-r--r-- 1 root root 272K Jan  3 20:19 div_with_fips.csv
4.0K drwxr-xr-x 4 root root 4.0K Jan  3 20:19 DWBI_ETL
240K -rw-r--r-- 1 root root 237K Jan  3 20:19 Income_fips.csv
428K -rw-r--r-- 1 root root 428K Jan  3 23:00 la.area.txt
114M -rw-r--r-- 1 root root 114M Jan  3 23:00 la.data.0.CurrentU10-14.txt
114M -rw-r--r-- 1 root root 114M Jan  3 23:00 la.data.0.CurrentU15-19.txt
216K -rw-r--r-- 1 root root 216K Jan  3 20:19 revenu.csv
4.0K drwxr-xr-x 1 root root 4.0K Dec 21 17:29 sample_data
384K -rw-r--r-- 1 root root 384K Jan  3 20:19 Table_crime.csv
280K -rw-r--r-- 1 root root 279K Jan  3 20:19 Table_div.csv
 22M -rw-r--r-- 1 root root  22M Jan  3 20:19 Table_employment.csv
1.3M -rw-r--r-- 1 root root 1.3M Jan  3 20:19 Table_population.csv
248K -rw-r--r-- 1 root root 246K Jan  3 20:19 Table_revenu.csv


Import Other Dataset from Repo

In [74]:
!rm -R DWBI_ETL
!git clone https://github.com/RedaElmar/DWBI_ETL

Cloning into 'DWBI_ETL'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 29 (delta 4), reused 11 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), done.


In [75]:
import pandas as pd
import requests
data=pd.read_csv("BLS_county_employment.csv")

In [76]:
data

,FIPS,state,countyname,year,month,Employment,Labor_Force,Unemployment,Unemployment_Rate
0,1001,AL,Autauga County,2010,1,22417.0,25072.0,2655.0,10.6
1,1001,AL,Autauga County,2010,2,22522.0,25051.0,2529.0,10.1
2,1001,AL,Autauga County,2010,3,22691.0,25071.0,2380.0,9.5
3,1001,AL,Autauga County,2010,4,23632.0,25829.0,2197.0,8.5
4,1001,AL,Autauga County,2010,5,23832.0,25931.0,2099.0,8.1
...,...,...,...,...,...,...,...,...,...
386155,72153,PR,Yauco Municipio,2019,8,8430.0,9598.0,1168.0,12.2
386156,72153,PR,Yauco Municipio,2019,9,8273.0,9633.0,1360.0,14.1
386157,72153,PR,Yauco Municipio,2019,10,8312.0,9713.0,1401.0,14.4
386158,72153,PR,Yauco Municipio,2019,11,8240.0,9625.0,1385.0,14.4


In [77]:
len(data.FIPS.unique())

3218

In [78]:
df_diversity = pd.read_csv('DWBI_ETL/data/diversityindex.csv')

fix some enteries and normalise data

In [79]:
df_diversity["Location"] = df_diversity["Location"].replace(["Do?a Ana County, NM"],"Dona Ana County, NM")
df_diversity.columns = ["Location","Diversity","Black or AA","American Indian","Asian","Hawaiian","Two or More","Latino","White"]
df_diversity.head()

,Location,Diversity,Black or AA,American Indian,Asian,Hawaiian,Two or More,Latino,White
0,"Aleutians West Census Area, AK",0.769346,7.4,13.8,31.1,2.3,4.8,14.6,29.2
1,"Queens County, NY",0.742224,20.9,1.3,25.2,0.2,2.7,28.0,26.7
2,"Maui County, HI",0.740757,0.8,0.6,28.8,10.6,23.3,10.7,31.5
3,"Alameda County, CA",0.740399,12.4,1.2,28.2,1.0,5.2,22.7,33.2
4,"Aleutians East Borough, AK",0.738867,7.7,21.8,41.4,0.7,3.7,13.5,12.9


In [80]:
df_diversity.tail

<bound method NDFrame.tail of                             Location  Diversity  ...  Latino  White
0     Aleutians West Census Area, AK   0.769346  ...    14.6   29.2
1                  Queens County, NY   0.742224  ...    28.0   26.7
2                    Maui County, HI   0.740757  ...    10.7   31.5
3                 Alameda County, CA   0.740399  ...    22.7   33.2
4         Aleutians East Borough, AK   0.738867  ...    13.5   12.9
...                              ...        ...  ...     ...    ...
3190                Osage County, MO   0.037540  ...     0.6   98.1
3191              Lincoln County, WV   0.035585  ...     0.6   98.2
3192               Leslie County, KY   0.035581  ...     0.5   98.2
3193               Blaine County, NE   0.023784  ...     0.2   98.8
3194            Keya Paha County, NE   0.021816  ...     0.5   98.9

[3195 rows x 9 columns]>

In [81]:
df_crime = pd.read_csv('DWBI_ETL/data/crime_data_w_population_and_crime_rate.csv')


Creation a FIPS variable to Identify County and to use as ID for Joining

In [82]:
def addzeros(lenght,txt):
  txt =str(txt)
  while len(txt)<lenght:
    txt= "0"+txt
  return txt
df_crime.FIPS_CTY = df_crime.FIPS_CTY.apply(lambda x : addzeros(3,x) )


In [83]:
df_crime["FIPS"] = df_crime.FIPS_ST.astype(str) + df_crime.FIPS_CTY

In [84]:
df_crime.FIPS = df_crime.FIPS.astype(str)

In [85]:
df_crime

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,COVIND,INDEX,MODINDX,MURDER,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,FIPS
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,100.0,5706,22329,119,200,1778,3609,4995,13791,3543,464,318416,29,510,29510
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,100.0,873,3424,8,38,165,662,1482,1753,189,28,49746,5,035,5035
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,100.0,127,278,1,2,5,119,82,184,12,2,7629,17,003,17003
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,100.0,6,13,0,3,1,2,5,4,4,0,412,48,261,48261
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,100.0,392,703,3,4,17,368,149,494,60,0,27083,22,031,22031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,"Ohio County, IN",0.000000,3132,1,4,762,6084,0,2,1,100.0,0,5,0,0,0,0,2,2,0,0,5994,18,115,18115
3132,"Newton County, MS",0.000000,3133,1,4,1465,21545,3346,3,1,100.0,0,5,0,0,0,0,4,0,1,0,21689,28,101,28101
3133,"Jerauld County, SD",0.000000,3134,1,4,2424,2108,2108,1,1,100.0,0,5,0,0,0,0,1,3,1,0,2066,46,073,46073
3134,"Cimarron County, OK",0.000000,3135,1,4,2167,2502,2502,2,2,100.0,0,3,0,0,0,0,1,2,0,0,2335,40,025,40025


Using External Library to provide FIPS from County_name,State

In [86]:
!pip install addfips


In [87]:
import addfips
af = addfips.AddFIPS()
df_diversity['TEMcounty']=df_diversity['Location']
df_diversity['TEMstate']=df_diversity['Location']

In [88]:

df_diversity

,Location,Diversity,Black or AA,American Indian,Asian,Hawaiian,Two or More,Latino,White,TEMcounty,TEMstate
0,"Aleutians West Census Area, AK",0.769346,7.4,13.8,31.1,2.3,4.8,14.6,29.2,"Aleutians West Census Area, AK","Aleutians West Census Area, AK"
1,"Queens County, NY",0.742224,20.9,1.3,25.2,0.2,2.7,28.0,26.7,"Queens County, NY","Queens County, NY"
2,"Maui County, HI",0.740757,0.8,0.6,28.8,10.6,23.3,10.7,31.5,"Maui County, HI","Maui County, HI"
3,"Alameda County, CA",0.740399,12.4,1.2,28.2,1.0,5.2,22.7,33.2,"Alameda County, CA","Alameda County, CA"
4,"Aleutians East Borough, AK",0.738867,7.7,21.8,41.4,0.7,3.7,13.5,12.9,"Aleutians East Borough, AK","Aleutians East Borough, AK"
...,...,...,...,...,...,...,...,...,...,...,...
3190,"Osage County, MO",0.037540,0.3,0.2,0.1,0.0,0.7,0.6,98.1,"Osage County, MO","Osage County, MO"
3191,"Lincoln County, WV",0.035585,0.2,0.1,0.1,0.0,0.7,0.6,98.2,"Lincoln County, WV","Lincoln County, WV"
3192,"Leslie County, KY",0.035581,0.4,0.1,0.2,0.0,0.7,0.5,98.2,"Leslie County, KY","Leslie County, KY"
3193,"Blaine County, NE",0.023784,0.2,0.0,0.0,0.0,0.8,0.2,98.8,"Blaine County, NE","Blaine County, NE"


In [89]:
def takeElt(text,i):
  if "," in text:
    return text.split(", ")[i]

df_diversity['TEMcounty']=df_diversity['TEMcounty'].apply(lambda x: takeElt(x,0))
df_diversity['TEMstate']=df_diversity['TEMstate'].apply(lambda x: takeElt(x,1))

In [90]:
df_diversity.Location[df_diversity["Location"]=="Dona Ana County, NM" ]

825    Dona Ana County, NM
Name: Location, dtype: object

In [91]:
import addfips
af = addfips.AddFIPS()
af.get_county_fips('Dona Ana County', state='NM')


'35013'

In [92]:
df_diversity.to_csv('div.csv', index=False)
!addfips div.csv --state-field TEMstate --county-field TEMcounty > div_with_fips.csv


In [93]:
import numpy as np
df_divFips= pd.read_csv('div_with_fips.csv')


In [94]:
import math
df_divFips['fips'] =df_divFips['fips'].apply(lambda x : 0 if math.isnan(x) else int(x))

In [95]:
df_divFips

,fips,Location,Diversity,Black or AA,American Indian,Asian,Hawaiian,Two or More,Latino,White,TEMcounty,TEMstate
0,2016,"Aleutians West Census Area, AK",0.769346,7.4,13.8,31.1,2.3,4.8,14.6,29.2,Aleutians West Census Area,AK
1,36081,"Queens County, NY",0.742224,20.9,1.3,25.2,0.2,2.7,28.0,26.7,Queens County,NY
2,15009,"Maui County, HI",0.740757,0.8,0.6,28.8,10.6,23.3,10.7,31.5,Maui County,HI
3,6001,"Alameda County, CA",0.740399,12.4,1.2,28.2,1.0,5.2,22.7,33.2,Alameda County,CA
4,2013,"Aleutians East Borough, AK",0.738867,7.7,21.8,41.4,0.7,3.7,13.5,12.9,Aleutians East Borough,AK
...,...,...,...,...,...,...,...,...,...,...,...,...
3190,29151,"Osage County, MO",0.037540,0.3,0.2,0.1,0.0,0.7,0.6,98.1,Osage County,MO
3191,54043,"Lincoln County, WV",0.035585,0.2,0.1,0.1,0.0,0.7,0.6,98.2,Lincoln County,WV
3192,21131,"Leslie County, KY",0.035581,0.4,0.1,0.2,0.0,0.7,0.5,98.2,Leslie County,KY
3193,31009,"Blaine County, NE",0.023784,0.2,0.0,0.0,0.0,0.8,0.2,98.8,Blaine County,NE


In [96]:
pop_data=pd.read_csv("DWBI_ETL/data/PopulationEstimates.csv", encoding = "ISO-8859-1")
pop_data

,FIPStxt,State,Area_Name,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,POP_ESTIMATE_2010,POP_ESTIMATE_2011,POP_ESTIMATE_2012,POP_ESTIMATE_2013,POP_ESTIMATE_2014,POP_ESTIMATE_2015,POP_ESTIMATE_2016,POP_ESTIMATE_2017,POP_ESTIMATE_2018,POP_ESTIMATE_2019,N_POP_CHG_2010,N_POP_CHG_2011,N_POP_CHG_2012,N_POP_CHG_2013,N_POP_CHG_2014,N_POP_CHG_2015,N_POP_CHG_2016,N_POP_CHG_2017,N_POP_CHG_2018,N_POP_CHG_2019,Births_2010,Births_2011,Births_2012,Births_2013,Births_2014,Births_2015,Births_2016,Births_2017,Births_2018,Births_2019,...,R_death_2016,R_death_2017,R_death_2018,R_death_2019,R_NATURAL_INC_2011,R_NATURAL_INC_2012,R_NATURAL_INC_2013,R_NATURAL_INC_2014,R_NATURAL_INC_2015,R_NATURAL_INC_2016,R_NATURAL_INC_2017,R_NATURAL_INC_2018,R_NATURAL_INC_2019,R_INTERNATIONAL_MIG_2011,R_INTERNATIONAL_MIG_2012,R_INTERNATIONAL_MIG_2013,R_INTERNATIONAL_MIG_2014,R_INTERNATIONAL_MIG_2015,R_INTERNATIONAL_MIG_2016,R_INTERNATIONAL_MIG_2017,R_INTERNATIONAL_MIG_2018,R_INTERNATIONAL_MIG_2019,R_DOMESTIC_MIG_2011,R_DOMESTIC_MIG_2012,R_DOMESTIC_MIG_2013,R_DOMESTIC_MIG_2014,R_DOMESTIC_MIG_2015,R_DOMESTIC_MIG_2016,R_DOMESTIC_MIG_2017,R_DOMESTIC_MIG_2018,R_DOMESTIC_MIG_2019,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018,R_NET_MIG_2019
0,0,US,United States,NaN,NaN,NaN,NaN,NaN,"308,745,538","308,758,105","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523","563,561","2,235,208","2,274,116","2,162,725","2,307,293","2,334,155","2,306,148","2,044,228","1,701,962","1,552,022","987,836","3,973,485","3,936,976","3,940,576","3,963,195","3,992,376","3,962,654","3,901,982","3,824,521","3,791,712",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,NaN,"4,779,736","4,780,125","4,785,437","4,799,069","4,815,588","4,830,081","4,841,799","4,852,347","4,863,525","4,874,486","4,887,681","4,903,185","5,312","13,632","16,519","14,493","11,718","10,548","11,178","10,961","13,195","15,504","14,226","59,690","59,067","57,929","58,903","59,647","59,389","58,961","58,271","57,313",...,10.6,10.9,11.0,11.0,2.3,2.2,1.5,1.9,1.6,1.6,1.2,0.9,0.7,1.0,1.2,1.0,0.8,0.9,1.2,0.6,0.7,0.6,-0.4,0.0,0.5,-0.2,-0.3,-0.4,0.5,1.1,1.9,0.6,1.2,1.5,0.6,0.6,0.7,1.1,1.8,2.5
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,0.0,"54,571","54,597","54,773","55,227","54,954","54,727","54,893","54,864","55,243","55,390","55,533","55,869",176,454,-273,-227,166,-29,379,147,143,336,150,638,615,571,640,651,666,676,631,624,...,9.9,10.4,9.3,9.7,2.3,1.0,-0.2,1.2,1.2,2.2,1.9,2.0,1.5,0.1,-0.3,0.2,0.1,0.2,-0.1,-0.2,-0.1,-0.3,5.9,-6.0,-4.1,1.8,-1.9,4.8,1.1,0.7,4.8,6.0,-6.2,-3.9,2.0,-1.7,4.8,0.8,0.5,4.6
3,1003,AL,Baldwin County,4.0,3.0,5.0,2.0,5.0,"182,265","182,265","183,112","186,558","190,145","194,885","199,183","202,939","207,601","212,521","217,855","223,234",847,"3,446","3,587","4,740","4,298","3,756","4,662","4,920","5,334","5,379",516,"2,189","2,093","2,160","2,212","2,257","2,300","2,300","2,310","2,304",...,9.9,10.0,10.7,10.5,1.9,1.1,1.4,1.1,0.8,1.4,1.0,0.0,-0.1,1.0,1.3,1.1,0.6,0.7,0.9,0.4,0.5,0.4,15.7,16.2,21.7,19.6,17.1,20.4,22.0,24.3,24.0,16.6,17.5,22.8,20.2,17.7,21.3,22.4,24.7,24.4
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,3.0,"27,457","27,455","27,327","27,341","27,169","26,937","26,755","26,283","25,806","25,157","24,872","24,686",-128,14,-172,-232,-182,-472,-477,-649,-285,-186,71,331,300,282,264,271,276,280,263,256,...,10.8,11.6,13.2,12.6,0.3,0.5,-0.5,-1.6,-2.3,-0.2,-0.6,-2.6,-2.3,-0.2,-0.4,-0.4,0.1,0.5,0.7,0.5,0.5,0.5,0.5,-6.5,-7.8,-5.3,-16.2,-18.9,-25.5,-9.2,-5.7,0.3,-6.9,-8.1,-5.1,-15.7,-18.2,-25.0,-8.8,-5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

Keeping only Relevent Data Columns 

In [97]:

pop_data=pop_data[["FIPStxt","State","Area_Name","POP_ESTIMATE_2010","POP_ESTIMATE_2011","POP_ESTIMATE_2012","POP_ESTIMATE_2013",
                  "POP_ESTIMATE_2014",	"POP_ESTIMATE_2015","POP_ESTIMATE_2016","POP_ESTIMATE_2017","POP_ESTIMATE_2018","POP_ESTIMATE_2019"]]

In [98]:
pop_data=pop_data.melt(id_vars=["FIPStxt","State","Area_Name"],var_name="year",value_name="pop_Estimate")

In [99]:
pop_data.year=pop_data.year.apply(lambda x : int(x[-4:])) 

In [100]:
pop_data

,FIPStxt,State,Area_Name,year,pop_Estimate
0,0,US,United States,2010,"309,321,666"
1,1000,AL,Alabama,2010,"4,785,437"
2,1001,AL,Autauga County,2010,"54,773"
3,1003,AL,Baldwin County,2010,"183,112"
4,1005,AL,Barbour County,2010,"27,327"
...,...,...,...,...,...
32725,72145,PR,"Vega Baja Municipio, Puerto Rico",2019,"50,023"
32726,72147,PR,"Vieques Municipio, Puerto Rico",2019,"8,386"
32727,72149,PR,"Villalba Municipio, Puerto Rico",2019,"21,372"
32728,72151,PR,"Yabucoa Municipio, Puerto Rico",2019,"32,282"


Normalise FIPS among all tables with County Level Data

In [101]:
def normalise_fips(x):
  while len(x)<6:
    x='0'+x
  return x
df_divFips['fips'] =df_divFips['fips'].apply(lambda x : "NaN" if math.isnan(x) else normalise_fips(str(x)))


In [102]:
pop_data['FIPStxt']=pop_data['FIPStxt'].apply(lambda x : "NaN" if math.isnan(x) else normalise_fips(str(x)))

In [103]:
df_divFips['year']=2016

In [104]:
df_divFips

,fips,Location,Diversity,Black or AA,American Indian,Asian,Hawaiian,Two or More,Latino,White,TEMcounty,TEMstate,year
0,002016,"Aleutians West Census Area, AK",0.769346,7.4,13.8,31.1,2.3,4.8,14.6,29.2,Aleutians West Census Area,AK,2016
1,036081,"Queens County, NY",0.742224,20.9,1.3,25.2,0.2,2.7,28.0,26.7,Queens County,NY,2016
2,015009,"Maui County, HI",0.740757,0.8,0.6,28.8,10.6,23.3,10.7,31.5,Maui County,HI,2016
3,006001,"Alameda County, CA",0.740399,12.4,1.2,28.2,1.0,5.2,22.7,33.2,Alameda County,CA,2016
4,002013,"Aleutians East Borough, AK",0.738867,7.7,21.8,41.4,0.7,3.7,13.5,12.9,Aleutians East Borough,AK,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,029151,"Osage County, MO",0.037540,0.3,0.2,0.1,0.0,0.7,0.6,98.1,Osage County,MO,2016
3191,054043,"Lincoln County, WV",0.035585,0.2,0.1,0.1,0.0,0.7,0.6,98.2,Lincoln County,WV,2016
3192,021131,"Leslie County, KY",0.035581,0.4,0.1,0.2,0.0,0.7,0.5,98.2,Leslie County,KY,2016
3193,031009,"Blaine County, NE",0.023784,0.2,0.0,0.0,0.0,0.8,0.2,98.8,Blaine County,NE,2016


In [105]:
pop_data

,FIPStxt,State,Area_Name,year,pop_Estimate
0,000000,US,United States,2010,"309,321,666"
1,001000,AL,Alabama,2010,"4,785,437"
2,001001,AL,Autauga County,2010,"54,773"
3,001003,AL,Baldwin County,2010,"183,112"
4,001005,AL,Barbour County,2010,"27,327"
...,...,...,...,...,...
32725,072145,PR,"Vega Baja Municipio, Puerto Rico",2019,"50,023"
32726,072147,PR,"Vieques Municipio, Puerto Rico",2019,"8,386"
32727,072149,PR,"Villalba Municipio, Puerto Rico",2019,"21,372"
32728,072151,PR,"Yabucoa Municipio, Puerto Rico",2019,"32,282"


In [106]:
df_crime['FIPS']=df_crime['FIPS'].apply(lambda x : "NaN" if math.isnan(int(x)) else normalise_fips(str(x)))
df_crime['year']=2016

In [107]:
df_crime

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,COVIND,INDEX,MODINDX,MURDER,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY,FIPS,year
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,100.0,5706,22329,119,200,1778,3609,4995,13791,3543,464,318416,29,510,029510,2016
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,100.0,873,3424,8,38,165,662,1482,1753,189,28,49746,5,035,005035,2016
2,"Alexander County, IL",1664.700485,3,1,4,604,8040,8040,2,2,100.0,127,278,1,2,5,119,82,184,12,2,7629,17,003,017003,2016
3,"Kenedy County, TX",1456.310680,4,1,4,2681,444,444,1,1,100.0,6,13,0,3,1,2,5,4,4,0,412,48,261,048261,2016
4,"De Soto Parish, LA",1447.402430,5,1,4,1137,26971,26971,3,3,100.0,392,703,3,4,17,368,149,494,60,0,27083,22,031,022031,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,"Ohio County, IN",0.000000,3132,1,4,762,6084,0,2,1,100.0,0,5,0,0,0,0,2,2,0,0,5994,18,115,018115,2016
3132,"Newton County, MS",0.000000,3133,1,4,1465,21545,3346,3,1,100.0,0,5,0,0,0,0,4,0,1,0,21689,28,101,028101,2016
3133,"Jerauld County, SD",0.000000,3134,1,4,2424,2108,2108,1,1,100.0,0,5,0,0,0,0,1,3,1,0,2066,46,073,046073,2016
3134,"Cimarron County, OK",0.000000,3135,1,4,2167,2502,2502,2,2,100.0,0,3,0,0,0,0,1,2,0,0,2335,40,025,040025,2016


In [108]:
data['FIPS']=data['FIPS'].apply(lambda x : "NaN" if math.isnan(int(x)) else normalise_fips(str(x)))

In [109]:
data

,FIPS,state,countyname,year,month,Employment,Labor_Force,Unemployment,Unemployment_Rate
0,001001,AL,Autauga County,2010,1,22417.0,25072.0,2655.0,10.6
1,001001,AL,Autauga County,2010,2,22522.0,25051.0,2529.0,10.1
2,001001,AL,Autauga County,2010,3,22691.0,25071.0,2380.0,9.5
3,001001,AL,Autauga County,2010,4,23632.0,25829.0,2197.0,8.5
4,001001,AL,Autauga County,2010,5,23832.0,25931.0,2099.0,8.1
...,...,...,...,...,...,...,...,...,...
386155,072153,PR,Yauco Municipio,2019,8,8430.0,9598.0,1168.0,12.2
386156,072153,PR,Yauco Municipio,2019,9,8273.0,9633.0,1360.0,14.1
386157,072153,PR,Yauco Municipio,2019,10,8312.0,9713.0,1401.0,14.4
386158,072153,PR,Yauco Municipio,2019,11,8240.0,9625.0,1385.0,14.4


In [110]:
data.to_csv('Table_employment.csv', index=False)
df_crime.to_csv('Table_crime.csv', index=False)

In [111]:
duplicateDFRow = df_divFips[df_divFips.duplicated(['fips'])]


these FIPS refer only to state average  we remove these entries

In [112]:
duplicateDFRow

,fips,Location,Diversity,Black or AA,American Indian,Asian,Hawaiian,Two or More,Latino,White,TEMcounty,TEMstate,year
32,000000,CALIFORNIA,0.674524,6.6,1.7,14.1,0.5,3.7,38.4,39.0,NaN,NaN,2016
57,000000,TEXAS,0.641294,12.4,1.0,4.3,0.1,1.8,38.4,44.0,NaN,NaN,2016
62,000000,NEVADA,0.635404,9.0,1.6,8.1,0.7,3.9,27.5,52.2,NaN,NaN,2016
84,000000,DISTRICT OF COLUMBIA,0.614376,49.5,0.6,3.9,0.1,2.6,10.1,35.8,NaN,NaN,2016
91,000000,MARYLAND,0.612776,30.1,0.6,6.1,0.1,2.6,9.0,53.3,NaN,NaN,2016
97,000000,NEW MEXICO,0.608758,2.5,10.4,1.6,0.2,2.4,47.3,39.4,NaN,NaN,2016
109,000000,NEW JERSEY,0.601993,14.7,0.6,9.2,0.1,2.0,18.9,57.6,NaN,NaN,2016
116,000000,NEW YORK,0.600981,17.5,1.0,8.2,0.1,2.3,18.4,57.2,NaN,NaN,2016
125,000000,FLORIDA,0.597203,16.7,0.5,2.7,0.1,1.9,23.6,56.4,NaN,NaN,2016
138,000000,GEORGIA,0.590880,31.4,0.5,3.7,0.1,1.9,9.2,54.8,NaN,NaN,2016


In [113]:
df_divFips.drop(df_divFips[df_divFips['fips']=="000000"].index, inplace=True)


In [114]:
df_divFips

,fips,Location,Diversity,Black or AA,American Indian,Asian,Hawaiian,Two or More,Latino,White,TEMcounty,TEMstate,year
0,002016,"Aleutians West Census Area, AK",0.769346,7.4,13.8,31.1,2.3,4.8,14.6,29.2,Aleutians West Census Area,AK,2016
1,036081,"Queens County, NY",0.742224,20.9,1.3,25.2,0.2,2.7,28.0,26.7,Queens County,NY,2016
2,015009,"Maui County, HI",0.740757,0.8,0.6,28.8,10.6,23.3,10.7,31.5,Maui County,HI,2016
3,006001,"Alameda County, CA",0.740399,12.4,1.2,28.2,1.0,5.2,22.7,33.2,Alameda County,CA,2016
4,002013,"Aleutians East Borough, AK",0.738867,7.7,21.8,41.4,0.7,3.7,13.5,12.9,Aleutians East Borough,AK,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,029151,"Osage County, MO",0.037540,0.3,0.2,0.1,0.0,0.7,0.6,98.1,Osage County,MO,2016
3191,054043,"Lincoln County, WV",0.035585,0.2,0.1,0.1,0.0,0.7,0.6,98.2,Lincoln County,WV,2016
3192,021131,"Leslie County, KY",0.035581,0.4,0.1,0.2,0.0,0.7,0.5,98.2,Leslie County,KY,2016
3193,031009,"Blaine County, NE",0.023784,0.2,0.0,0.0,0.0,0.8,0.2,98.8,Blaine County,NE,2016


In [115]:
duplicateDFRow = df_divFips[df_divFips.duplicated(['fips'])]
duplicateDFRow

,fips,Location,Diversity,Black or AA,American Indian,Asian,Hawaiian,Two or More,Latino,White,TEMcounty,TEMstate,year


same for revenu

In [117]:
df_revenu = pd.read_csv('DWBI_ETL/data/2015 Median Income by County.csv')

df_revenu["County"] = df_revenu["County"].replace(["Do��a Ana County	"],"Dona Ana County")
df_revenu.to_csv('revenu.csv', index=False)

In [118]:
!addfips 'revenu.csv' --state-field State --county-field County > Income_fips.csv


In [119]:
df_revenu_fips = pd.read_csv('Income_fips.csv')
df_revenu_fips

,fips,County-State,County,Population,Median household income,State Code,State
0,1001.0,"Autauga County, Alabama",Autauga County,55221,51281.0,AL,Alabama
1,1003.0,"Baldwin County, Alabama",Baldwin County,195121,50254.0,AL,Alabama
2,1005.0,"Barbour County, Alabama",Barbour County,26932,32964.0,AL,Alabama
3,1007.0,"Bibb County, Alabama",Bibb County,22604,38678.0,AL,Alabama
4,1009.0,"Blount County, Alabama",Blount County,57710,45813.0,AL,Alabama
...,...,...,...,...,...,...,...
3215,NaN,"Vega Baja Municipio, Puerto Rico",Vega Baja Municipio,56858,16948.0,WY,Wyoming
3216,NaN,"Vieques Municipio, Puerto Rico",Vieques Municipio,9130,18104.0,WY,Wyoming
3217,NaN,"Villalba Municipio, Puerto Rico",Villalba Municipio,24685,17818.0,WY,Wyoming
3218,NaN,"Yabucoa Municipio, Puerto Rico",Yabucoa Municipio,36279,15627.0,WY,Wyoming


In [120]:
df_revenu_fips.dropna(subset = ["fips"], inplace=True)

In [121]:
df_revenu_fips['fips']=df_revenu_fips['fips'].astype(float).astype(int).apply(lambda x : normalise_fips(str(x)))


In [122]:
df_revenu_fips['year']=2016

In [123]:
df_revenu_fips

,fips,County-State,County,Population,Median household income,State Code,State,year
0,001001,"Autauga County, Alabama",Autauga County,55221,51281.0,AL,Alabama,2016
1,001003,"Baldwin County, Alabama",Baldwin County,195121,50254.0,AL,Alabama,2016
2,001005,"Barbour County, Alabama",Barbour County,26932,32964.0,AL,Alabama,2016
3,001007,"Bibb County, Alabama",Bibb County,22604,38678.0,AL,Alabama,2016
4,001009,"Blount County, Alabama",Blount County,57710,45813.0,AL,Alabama,2016
...,...,...,...,...,...,...,...,...
3137,056037,"Sweetwater County, Wyoming",Sweetwater County,44772,69022.0,WY,Wyoming,2016
3138,056039,"Teton County, Wyoming",Teton County,22311,75325.0,WY,Wyoming,2016
3139,056041,"Uinta County, Wyoming",Uinta County,20930,56569.0,WY,Wyoming,2016
3140,056043,"Washakie County, Wyoming",Washakie County,8400,47652.0,WY,Wyoming,2016


In [124]:
df_revenu_fips.to_csv('Table_revenu.csv', index=False)

All fips with '000' in the end represents state average Data 

In [125]:
L=[normalise_fips(str(x)+"000") for x in range(0,52)]
L

['000000',
 '001000',
 '002000',
 '003000',
 '004000',
 '005000',
 '006000',
 '007000',
 '008000',
 '009000',
 '010000',
 '011000',
 '012000',
 '013000',
 '014000',
 '015000',
 '016000',
 '017000',
 '018000',
 '019000',
 '020000',
 '021000',
 '022000',
 '023000',
 '024000',
 '025000',
 '026000',
 '027000',
 '028000',
 '029000',
 '030000',
 '031000',
 '032000',
 '033000',
 '034000',
 '035000',
 '036000',
 '037000',
 '038000',
 '039000',
 '040000',
 '041000',
 '042000',
 '043000',
 '044000',
 '045000',
 '046000',
 '047000',
 '048000',
 '049000',
 '050000',
 '051000']

In [126]:
for n in L:
  pop_data.drop(pop_data[pop_data['FIPStxt']==n].index, inplace=True)
pop_data

,FIPStxt,State,Area_Name,year,pop_Estimate
2,001001,AL,Autauga County,2010,"54,773"
3,001003,AL,Baldwin County,2010,"183,112"
4,001005,AL,Barbour County,2010,"27,327"
5,001007,AL,Bibb County,2010,"22,870"
6,001009,AL,Blount County,2010,"57,376"
...,...,...,...,...,...
32725,072145,PR,"Vega Baja Municipio, Puerto Rico",2019,"50,023"
32726,072147,PR,"Vieques Municipio, Puerto Rico",2019,"8,386"
32727,072149,PR,"Villalba Municipio, Puerto Rico",2019,"21,372"
32728,072151,PR,"Yabucoa Municipio, Puerto Rico",2019,"32,282"


In [127]:
pop_data.to_csv('Table_population.csv', index=False, sep=';')

In [128]:
len(data["FIPS"].unique())

3218

In [134]:
len(df_revenu_fips["fips"].unique())

3140

In [130]:
len(df_crime["FIPS"].unique())

3136

In [131]:
len(df_divFips["fips"].unique())

3143

In [132]:
len(pop_data["FIPStxt"].unique())

3225

re save with different separator

 ( comma separated causing errors on import to SQL Server 'some entries contains comma')

In [135]:
pop_data.to_csv('Table_population.csv', index=False, sep=';')
df_revenu_fips.to_csv('Table_revenu.csv', index=False, sep=';')
df_divFips.to_csv('Table_div.csv', index=False, sep=';')
data.to_csv('Table_employment.csv', index=False, sep=';')
df_crime.to_csv('Table_crime.csv', index=False, sep=';')